In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import torch
import numpy as np

from trainer import load_embeddings
from lib.tokenizer import preprocessor
from lib.config import MODEL_EC, DEVICE
from lib.data_utils import vectorize

In [4]:
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

In [24]:
labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust', 'neutral']
model_path = '/home/houyu/learning/FinalProject/out/model/EmotionClassification_0.5900_2019-05-06_00:51.model'
model_conf = MODEL_EC
max_length = 65  # 85 train 65 dev 58 test
dev_path = '/home/houyu/learning/FinalProject/datasets/new/E-c-En-dev.txt'

# Loading model
model = torch.load(model_path)
model.eval()
word2idx, _, embeddings = load_embeddings(model_conf)

head = 1
cnt = 0
with open(dev_path, 'r') as fin:
    for line in fin.readlines():
        if head == 1:
            head = 0
            continue
        
        items = line[:len(line)-1].split('\t')
        
        input_text = items[1]

        # Pre-processing inputs
        pro_sent = preprocessor(input_text)

        # Embedding and vectorize
        sample = vectorize(pro_sent, word2idx, max_length)

        # Processing to get model inputs
        samples = []
        lengths = []
        samples.append(sample)
        lengths.append(len(pro_sent))

        samples = np.asarray(samples)
        lengths = np.asarray(lengths)

        samples = torch.tensor(samples)
        lengths = torch.tensor(lengths)

        samples = samples.to(DEVICE)
        lengths = lengths.to(DEVICE)


        # Running model
        outputs, attentions = model(samples, lengths)

        posts = outputs.data.cpu().numpy()
        # predicted = np.clip(np.sign(posts), a_min=0, a_max=None)    # 1   1   0   0   0   0   1   0   0   0   1
        # predicted = predicted.astype(np.int32)
        
        pro_sigmoid = sigmoid(posts)
        predicted = np.clip(np.sign(pro_sigmoid - 0.5), a_min=0, a_max=None)
        predicted = predicted.astype(np.int32)
        
        gold_labels = []
        for item in items[2:13]:
            gold_labels.append(int(item))
        
        gold_labels = np.asarray(gold_labels)
        
        acc = 0
        and_ = (predicted & gold_labels).sum()
        or_ = (predicted | gold_labels).sum()
        if(or_ != 0):
            acc = and_/or_
        
        # print(predicted)
        # print(gold_labels)
        # print(acc)
        # pro_sigmoid = sigmoid(posts)
        
        # break
        # if acc >= 0.98:            # <= 0.5 错误分析
            # print(input_text)
            # print(pro_sent)
            # print(attentions)
            # print(posts)
            # print(pro_sigmoid)
            # print(gold_labels)
            # cnt += 1

# cnt
        # print(pro_sent)
        # print(attentions)
        atten_array = attentions.data.cpu().numpy()
        # print(atten_array)
        # 1.0 only one value
        if len(pro_sent) == 1:
            atten = atten_array.tolist()
            atten_array = []
            atten_array.append(atten)
        # print(atten_array)
        atten_set = set(atten_array)
        # print(atten_set)
        if len(atten_set)/len(pro_sent) >= 0.3 and acc >= 0.98:
            print(acc)
            print(input_text)
            print(pro_sent)
            print(atten_array)
            print(posts)
            print(gold_labels)

Loading word embeddings: ntua_twitter_affect_310 ...
Loaded word embeddings from cache.
1.0
@CNN thanks Hollywood, (Depp). all these climate change fanatics using private jets and gas guzzling cars are to blame for this! #outrage
['<user>', 'thanks', 'hollywood', ',', '(', 'depp', ')', '.', 'all', 'these', 'climate', 'change', 'fanatics', 'using', 'private', 'jets', 'and', 'gas', 'guzzling', 'cars', 'are', 'to', 'blame', 'for', 'this', '!', '<hashtag>', 'outrage', '</hashtag>']
[ 0.00568191  0.00586119  0.00568194  0.00571045  0.00569288  0.00606835
  0.04165663  0.04198369  0.0419839   0.04198395  0.04198395  0.04198395
  0.04198395  0.04198395  0.04198395  0.04198395  0.04198395  0.04198395
  0.04198395  0.04198395  0.04198395  0.04198395  0.04198395  0.04198395
  0.04198395  0.04198395  0.04198395  0.04198395  0.04198395]
[  7.47934198  -5.45602512   3.8165741   -6.14770746  -6.02395487
 -10.53802299  -6.63674688  -4.34410477  -1.12866223  -6.12589216
 -10.19590855]
[1 0 1 0 0 0 0 0

In [23]:
cnt # acc >= 0.9; 0.5 245, 0.3 191 => first model, # new model 252

191

In [20]:
# 448 <= 0.5

In [25]:
a = {1:[]}

In [26]:
a = set()
a.add(1)
a.add(1)
a

{1}